In [ ]:
import sys
import time
from dataclasses import dataclass
from PIL import Image, ImageDraw

sys.path.append('../utils')
from pyutils import *

In [ ]:
@dataclass
class Edge:
    path: list[Pt]
    dead: bool = False

In [ ]:
def scan_vertices(mat: StrMatrix) -> dict[Pt, dict[Pt, Edge]]:
    verts: dict[Pt, dict[Pt, Edge]] = {}
    for pt, val in mat_iter(mat):
        if val == '#':
            continue
        branches: list[Pt] = []
        for direc in Pt.cardinals():
            future = pt + direc
            if matget(mat, future) != '#':
                branches.append(future)
        if len(branches) > 2:
            verts[pt] = {br:Edge([pt]) for br in branches}

    return verts

In [ ]:
def build_graph(maze: Matrix, start: Pt, end: Pt, gif: list[Image.Image] | None = None) -> dict[Pt, dict[Pt, Edge]]:
    verts: dict[Pt, dict[Pt, Edge]] = scan_vertices(maze)
    if start not in verts:
        verts[start] = {start + br:Edge([start]) for br in Pt.cardinals() if matget(maze, start + br) != '#'}
    if end not in verts:
        verts[end] = {end + br:Edge([end]) for br in Pt.cardinals() if matget(maze, end + br) != '#'}

    def explore_until_vert(head: Pt, edge: Edge):
        cursor: Pt = head
        while True:
            edge.path.append(cursor)
            if cursor in verts:
                return
            branches: list[Pt] = []
            for br in Pt.cardinals():
                future = cursor + br
                if (matget(maze, future) != '#') and (future not in edge.path):
                    branches.append(cursor + br)
            if len(branches) == 0:
                edge.dead = True
                return
            if len(branches) == 1:
                cursor = branches[0]
                continue

    cursor: Pt = start
    seen: set[Pt] = set()
    for k in verts:
        for head, edge in verts[k].items():
            explore_until_vert(head, edge)
            seen.update(edge.path)
            if gif is not None:
                gif.append(matimg(maze, colflt={
                    (lambda p,v: v == '#'): 'black',
                    (lambda p,v: p in seen): 'grey',
                    (lambda p,v: p in edge.path): 'red',
                    (lambda p,v: p in verts): 'blue',
                    (lambda p,v: p == k): 'magenta'
                }))

    return verts

In [ ]:
def path_astar(graph: dict[Pt, dict[Pt, Edge]], start: Pt, end: Pt) -> list[Pt]:
    vertstack: list[Pt] = []

    vert: Pt = start
    g_total: int = 0
    while True:
        # print()
        # time.sleep(0.5)
        # print(vert, g_total, vertstack)
        edges: dict[Pt, Edge] = {h:e for h, e in graph[vert].items() if (not e.dead) and (e.path[-1] not in vertstack)}
        # print('.', vert, edges)
        scores: dict[Pt, int] = {}
        for head, edge in edges.items():
            # print('...', head, edge)
            tail: Pt = edge.path[-1]
            if tail == end:
                vertstack.append(tail)
                return vertstack
            g: int = g_total + len(edge.path)
            # h: int = tail.distance(end)
            h = 0
            # print('...##', head, tail, g, h, g + h)
            scores[head] = g + h
        vertstack.append(vert)
        best = edges[sorted(edges, key=lambda k: scores[k])[0]].path
        vert = best[-1]
        g_total += len(best)

In [ ]:
sample = readutf8('sample.txt')
real = readutf8('input.txt')
maze = strtomat(real)

In [ ]:
start, end = Pt(len(maze) - 2, 1), Pt(1, len(maze[0]) - 2)

In [ ]:
ta = time.perf_counter()
_frames = []
graph = build_graph(maze, start, end)
tb = time.perf_counter()
print(f'{tb - ta:.8f}')
print(len(graph), f'@ {sys.getsizeof(graph) / 1000}KB')

In [ ]:
# _frames[0].save('out.gif', save_all=True, append_images=_frames[1:], duration=10)

In [ ]:
ret = path_astar(graph, start, end)

In [ ]:
ret

In [ ]:
matimg(maze, colflt={
    (lambda p,v: v == '#'): 'black',
    (lambda p,v: p in ret): 'blue'
}, resize=(500,500))